In [1]:
# TO BHMA

from bs4 import BeautifulSoup
import requests

# Get the base URL and tag name from user input
url = input('give the URL please')
url = url + 'page/1/'  # Appends the first page path to the URL
tname = input('print the tag name')

# Construct filenames for CSV outputs
a = 'To_Vima'
b = '_Wdate'
csv = '.csv'
simple = a+tname+csv
wd = a+tname+b+csv

# Lists to store tags and publication dates
tag_list = []
date_list = []

# Date threshold to stop the scraping
stop_date = 20241231

# Function to update the page number in the URL
def new_url():
    global url 
    splitted_url = url.split('/')
    nr = int(splitted_url[6])+1  # Increments the page number
    splitted_url[6] = str(nr)
    url = "/".join(splitted_url)  # Joins back the URL with new page number

# Main scraping function
def main_fun(stop_date, url):
    global tag_list
    global date_list
    global flag

    # Function to extract tags from an individual article
    def tag_function(inside):
        global tag_list
        inside_tag_list = []
        page = requests.get(inside)
        soup = BeautifulSoup(page.content, 'html.parser')

        try:
            # Tries to extract the top/main tag
            upper_tag = soup.find('div' , class_ = 'wrap-article-header')
            upper_tag = upper_tag.find('a')
            upper_tag = upper_tag.get_text()
            inside_tag_list.append(upper_tag)
        except AttributeError:
            pass  # If tag not found, skip

        # Extracts the lower tags listed in the article
        tag_section = soup.find('div' , class_ = 'article-column')
        tag_section = tag_section.find_all('li' , class_ = 'tag-item')
        for a in tag_section:
            tag  = a.get_text()
            inside_tag_list.append(tag)

        tag_list.append(inside_tag_list)

    # Fetch and parse the list page
    page = requests.get(url)
    soup = BeautifulSoup(page.content, 'html.parser')
    art_column = soup.find('div' , class_ = 'column is-relative')
    articles = art_column.find_all('a' , class_ = "columns is-mobile is-multiline")
    
    # Loop through the articles to extract the date and URL
    n = 0
    for article in articles:
        n+=1
        par_date = article.find('time')
        par_date = par_date.get_text()
        date = par_date.split(",")
        date = date[0]
        date = date.split(".")

        # Stops after 20 articles (fail-safe)
        if n==21:
            print('n occured')
            break

        # Cleans and reorders date
        nd=[]
        for dates in date:
            for i in dates:
                if i.isnumeric():
                    continue
                else:
                    i=""  # Unused, has no effect
            dates = dates.strip()
            nd.append(dates)
        nd.reverse()
        date = "".join(nd)
        date = int(date)

        # Checks if the date is older than the stop threshold
        if date < stop_date:
            print('date<stop_date')
            flag = False
            break
        else:
            date_list.append(date)
            article_url = article.get('href')
            https = "https://www.tovima.gr"
            if article_url.find(https)<0:
                print('vrika arthro spasmeno. To url tou einai ' , article_url)
                print('Please check the Url..')
            else:
                tag_function(article_url)  # Extract tags from the article

    # Stops if there are less than 20 articles in the page
    if len(articles) < 20:
        print('the nr of the articles in the page is < 20')
        flag = False

# Loop through pages while flag is True
flag = True
while flag:
    main_fun(stop_date,url)
    new_url()  # Move to next page

# Print summary information
print('the length of the taglist is ' , len(tag_list))
print('the length of the datelist is ' , len(date_list))
print(tag_list)
print('the last date is ' , date_list[-1])

import pandas as pd

# Flatten tag list
new_tag_list = []
for i in range(len(tag_list)):
    for j in range(len(tag_list[i])):
        new_tag_list.append(tag_list[i][j])

# Count occurrences of each tag
res = {}
for i in new_tag_list:
    res[i]=new_tag_list.count(i)
print(res)

# Convert dict to list of lists
resultList = new_list = list(map(list, res.items()))
print(resultList)

# Save tag frequency to CSV
df_costliness = pd.DataFrame(resultList)
df_costliness.to_csv(simple, header=False)

# Convert dates to dd.mm.yyyy format
ddate_list = date_list
ttag_list = tag_list
ndlist = []
for i in ddate_list:
    i = str(i)
    y = i[:4]
    m = i[4:6]
    d = i[6:]
    ddate = d + '.' + m + '.' + y
    ndlist.append(ddate)

print(ddate_list)
print(ndlist)

# Create a tag-date pair list for CSV
n = 0
date_con_list = []
for i in ttag_list:
    n += 1
    for tag in i:
        inside = []
        inside.append(tag)
        inside.append(ddate_list[n-1])
        date_con_list.append(inside)
print(date_con_list)

# Save tag-date pairs to CSV
df_connected = pd.DataFrame(date_con_list)
df_connected.to_csv(wd, header=False)

# Final printouts
print(len(tag_list))
print(len(date_list))
print(tag_list)
print('the last date collected is ' , date_list[-1])
print('the nr of the tags mentioned is ' , len(resultList))


date<stop_date
the length of the taglist is  93
the length of the datelist is  93
[['Πολιτική ', '  δημοσκόπηση ', '  Αντώνης Σαμαράς ', '  Αλέξης Τσίπρας ', '  Νέα Δημοκρατία ', '  ΠΑΣΟΚ ', '  ΠΛΕΥΣΗ ΕΛΕΥΘΕΡΙΑΣ ', '  ΚΚΕ ', '  ΣΥΡΙΖΑ ', '  ακρίβεια '], ['Πολιτική ', '  δημοσκόπηση ', '  Αλέξης Τσίπρας ', '  Ζωή Κωνσταντοπούλου ', '  ΣΥΡΙΖΑ ', '  ΠΑΣΟΚ ', '  Νέα Αριστερά ', '  ΠΛΕΥΣΗ ΕΛΕΥΘΕΡΙΑΣ ', '  κεντροαριστερά '], ['Κοινωνία ', '  δημοσκόπηση ', '  Νέα Δημοκρατία ', '  ΠΑΣΟΚ ', '  ΠΛΕΥΣΗ ΕΛΕΥΘΕΡΙΑΣ ', '  ΣΥΡΙΖΑ ', '  Ελληνική Λύση ', '  ακρίβεια ', '  μεταναστευτικό ', '  εγκληματικότητα '], ['Πολιτική ', '\n\n          Κυριάκος Μητσοτάκης        \n', '\n\n          ΟΠΕΚΕΠΕ        \n', '\n\n          Ευρωάμυνα        \n', '\n\n          προανακριτική        \n', '\n\n          Τέμπη        \n', '\n\n          τραγωδία στα Τέμπη        \n', '\n\n          Νέα Δημοκρατία        \n', '\n\n          κυβέρνηση        \n', '\n\n          Κώστας Αχ. Καραμανλής        \n', '\n\n          